## Data preprocessing

In [1]:
import numpy as np
import pandas as pd

colnames = ["time", "ID", "DLC", "Data1", \
        "Data2", "Data3", "Data4", "Data5", "Data6", "Data7", "Data8", "Attack"]

nRows = 10000 #number of rows that you want
df = pd.read_csv('gear_dataset.csv', nrows = nRows, sep=',', names=colnames)

uniqueIDs = df['ID'].unique() #26 for the entire dataset

#Drop attack packets
attack = df[df['Attack'] == 'T'].copy()
df.drop(attack.index, axis=0, inplace=True)

#Drop DLC = 2 packets
dlc2 = df[df['DLC'] == 2]
df.drop(dlc2.index, axis=0, inplace=True) #drop all dlc2 indexes

#Reset index from 1 to n (not needed actually, so commenting out)
#df.set_index(np.arange(len(df)), inplace=True)

#Pick an ID
id_data= df[df['ID'] == '0140'].copy()

#Just use data values without time, Attack, ID and DLC right now
dataValues = id_data.drop(["time", "Attack", "ID", "DLC"], axis = 1).copy()
dataValues = dataValues.to_numpy()


storage = np.zeros((len(dataValues),64), dtype=int)
for currentRow in np.arange(len(storage)):
    
    tempString = "".join(dataValues[currentRow])
    formatted = format(int(tempString, base=16), "064b")
    storage[currentRow,:] = np.array(list(formatted), dtype=int)

In [9]:
print(storage.shape)
n_rows = storage.shape[0]
n_features = storage.shape[1]

(420, 64)


In [6]:
from matplotlib import pyplot
import keras
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
import numpy as np
from numpy import array


In [2]:
#A = array([0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0])
A = array([0,1,1,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1])
#B = A[::-1] # not sure if this is the way to do it (should it be just each sample that is reversed, or the whole sequence?)

#A = A.reshape((5, 4, 1))
# for x in A:
#     C = A[::-1] 

time_steps = 4
n_features = 1
#n_samples = A.shape[0]
#C = np.flip(A,1)


In [44]:

def overlapping_window (window_size,overlap,seq): # overlap 1 is max. larger number would be less overlap
 
    seq = array([seq[i : i + window_size] for i in range(0, len(seq), overlap)]) 
   
    correct = [len(x)==window_size for x in seq]
    seq = seq[correct]
    seq = np.stack(seq, axis=0 )
    seq = seq.reshape(-1,window_size,1)

    return seq



In [62]:
time_steps = 40
a = np.r_[0:n_rows]
X_train_samples = overlapping_window(time_steps,20,a)
X_train = storage[X_train_samples,:]
X_train = np.squeeze(X_train)
print(X_train.shape)


(20, 40, 64)
<ipython-input-44-adff2931e955>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq = array([seq[i : i + window_size] for i in range(0, len(seq), overlap)])


In [63]:
# time_steps = 40
# n_features = 64
# 64*40 = 2560 output shape??
n_samples = X_train.shape[0]
time_steps = X_train.shape[1]
n_features = X_train.shape[2]

lstm_initializer = tf.keras.initializers.RandomUniform(minval=-0.08, maxval=0.08)

AutoEncoder = Sequential()

# Encoder
AutoEncoder.add(Dense(256, activation='tanh', input_shape=(time_steps,n_features)))
AutoEncoder.add(Dropout(0.2))

AutoEncoder.add(LSTM(128, return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer))
AutoEncoder.add(LSTM(128, return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer))
# Decoder
AutoEncoder.add(LSTM(128, return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer))
AutoEncoder.add(LSTM(128,return_sequences=True,kernel_initializer =lstm_initializer, recurrent_initializer=lstm_initializer))

AutoEncoder.add(Dense(256, activation= 'sigmoid'))
AutoEncoder.add(Dense(1, activation= 'sigmoid')) # they don't mention this in the article but dimensions are wrong otherwise

AutoEncoder.compile(optimizer='adam', loss='binary_crossentropy')
AutoEncoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40, 256)           16640     
_________________________________________________________________
dropout (Dropout)            (None, 40, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 40, 128)           197120    
_________________________________________________________________
lstm_1 (LSTM)                (None, 40, 128)           131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 40, 128)           131584    
_________________________________________________________________
dense_1 (Dense)              (None, 40, 256)           3

In [64]:
model = AutoEncoder
es= EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)

history = model.fit(X_train, X_train, epochs=1000, verbose=0, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.legend()
pyplot.show()

ValueError: in user code:

    C:\Users\ellsan\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\ellsan\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ellsan\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ellsan\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ellsan\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ellsan\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step

In [10]:
output = AutoEncoder.predict(A_window) #  Much better when i don't reverse the sequences? reversed incorrectly?
print(output)
print(output.shape)

[[[1.2522936e-04]
  [9.9987435e-01]
  [9.9989957e-01]
  [3.5602118e-06]]

 [[9.9989343e-01]
  [6.6380248e-06]
  [9.9988598e-01]
  [4.7716467e-06]]

 [[9.9989343e-01]
  [6.6380248e-06]
  [6.1857027e-06]
  [2.8600830e-06]]

 [[1.2522936e-04]
  [1.3947010e-05]
  [5.9488966e-06]
  [3.9616484e-06]]

 [[1.2522936e-04]
  [1.3947010e-05]
  [9.9981093e-01]
  [9.9989963e-01]]

 [[9.9989343e-01]
  [9.9989951e-01]
  [9.9989963e-01]
  [2.3557363e-06]]

 [[9.9989343e-01]
  [6.6380248e-06]
  [6.1856972e-06]
  [9.9987495e-01]]

 [[1.2522936e-04]
  [9.9987435e-01]
  [8.2043825e-06]
  [3.7635368e-06]]

 [[1.2522936e-04]
  [1.3947010e-05]
  [9.9981093e-01]
  [9.9989963e-01]]]
(9, 4, 1)


In [11]:
output2 = AutoEncoder.predict(A_window[None,1,:,:])


print(output2)
print(output2.shape)

[[[9.9989343e-01]
  [6.6380248e-06]
  [9.9988598e-01]
  [4.7716467e-06]]]
(1, 4, 1)


In [12]:
output[output<0.5] = 0
output[output>0.5] = 1
print(output)

[[[0.]
  [1.]
  [1.]
  [0.]]

 [[1.]
  [0.]
  [1.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  [1.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [1.]
  [1.]]]
